In [1]:
#Downloading necessary libraries
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score

In [2]:
#Loading training data from file
Trening = pd.read_csv("sc_project_training_data.csv")

In [3]:
#Loading test data from file
Test = pd.read_csv("sc_project_test_data.csv")

In [4]:
#Swaping data from column GameType to numeric
#New data - integers from 1 to 6 inclusive - saving in new column
df = pd.concat([Trening['GameType'], Test['GameType']])
layer = tf.keras.layers.StringLookup()
layer.adapt(df)
Trening['TypeofGame'] = layer(Trening['GameType'])
Test['TypeofGame'] = layer(Test['GameType'])

In [5]:
#Swaping data from column Winner to numeric
Trening['Wynik'] = 0
Trening.loc[Trening['Winner'] == 'A', 'Wynik'] = 1

In [6]:
#Deleting duplicated data and separating results from parameters in training set
#Receiving 3 sets, all with numeric data
#First one with training parameters
#Second one with information who won for training parameters
#Third one with test parameters
Test = Test.drop(columns = ['GameType'])
Wynik = Trening['Wynik']
Trening = Trening.drop(columns = ['GameID', 'Winner', 'GameType', 'Wynik'])

In [7]:
#Normalizing data
scaler = MinMaxScaler()
Trening_skalowany = scaler.fit_transform(Trening)
Test_skalowany = scaler.fit_transform(Test)

In [8]:
#Defining model
#model = Sequential()
#model.add(Dense(32, input_shape=(158,), activation='relu'))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(1, activation='sigmoid'))
model = keras.Sequential()
model.add(layers.Dense(32, input_shape=(158,), activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#The model is neural network made up of input layer, two hidden layers and output layer
#Size of input layer is 158 - it's the number of parameters, which result depends on
#Both hidden layers are the same - they are made up of 32 neuron each
#The function of activation is relu
#In the end there is input layer with function of activation sigmoid

In [9]:
#Compiling model
#As the function of loss we take binary crossentropy
#Optimizing with stochastic gradient descent
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['AUC'])

In [ ]:
#Training model
model.fit(Trening_skalowany, Wynik, epochs=100, batch_size=32)

Epoch 1/100
1074/1074 [==============================] - 6s 4ms/step - loss: 0.6659 - auc: 0.5679
Epoch 2/100
1074/1074 [==============================] - 8s 7ms/step - loss: 0.6496 - auc: 0.6439
Epoch 3/100
1074/1074 [==============================] - 4s 4ms/step - loss: 0.6331 - auc: 0.6597
Epoch 4/100
1074/1074 [==============================] - 5s 5ms/step - loss: 0.6249 - auc: 0.6665
Epoch 5/100
1074/1074 [==============================] - 3s 3ms/step - loss: 0.6219 - auc: 0.6699
Epoch 6/100
1074/1074 [==============================] - 4s 3ms/step - loss: 0.6200 - auc: 0.6722
Epoch 7/100
1074/1074 [==============================] - 4s 4ms/step - loss: 0.6182 - auc: 0.6752
Epoch 8/100
1074/1074 [==============================] - 3s 3ms/step - loss: 0.6172 - auc: 0.6768
Epoch 9/100
1074/1074 [==============================] - 3s 3ms/step - loss: 0.6165 - auc: 0.6772
Epoch 10/100
1074/1074 [==============================] - 3s 3ms/step - loss: 0.6158 - auc: 0.6781
Epoch 11/100
1074/1

In [ ]:
#Putting test set through the model
predictions = model.predict(Test_skalowany)

In [ ]:
#Saving results to text file
f = open("projekt_sgd_1.txt", "w")
for row in predictions:
    np.savetxt(f, row, fmt='%1.10f')
f.close()

In [ ]:
#Separating set to training set and validation set
#Prediction
dane_train, dane_test, wynik_train, wynik_test = train_test_split(Trening_skalowany, Wynik)
pred = model.predict(dane_test)
roc_auc_score(wynik_test, pred)

In [ ]:
#On 10% of test set we get conformity 0.6978 on average